In [10]:
import pandas as pd 
import glob
import xarray as xr
import time

In [106]:
path = "data/114"

more_files = sorted(glob.glob(path + "/*.dat"))[:20] 
#prøverunnde med noen filer. denne finner alle filene som slutterpå ".dat" og leser bare de 10 første
#ikke utvit til noe særlig mer en typ 20-30 stasjoner. da sprenger pcen. burde gå bedre når vi effektiviserer etterpå


first = True
start = time.time()
for i in more_files:
    
    df = pd.read_table(i,skiprows=2, delim_whitespace=True,skipfooter=1,engine="python")
    
    
    if(df.values.shape[0]<10): # 
        continue
    
    meta = pd.read_table(i,skiprows=None,sep="\s+",nrows=1,engine="python")
    print(df.values.shape[0], int(str(meta.head().columns[3])[:-1]))
    
    measurement_time = pd.to_datetime(float(meta.values[0,1]),origin=str(int(meta.values[0,0])),unit="D")
    measurement_lat  = float(meta.values[0,2])
    measurement_lon  = float(meta.values[0,3])
    #print(measurement_time)
    
    
    if "%year" in df.columns:
        df = df.drop(["%year","day"],axis=1)
    if "nobs" in df.columns:
        df = df.drop("nobs",axis=1)
    if "dissolved_oxygen" in df.columns:
        df = df.rename(columns={"dissolved_oxygen":"moles_of_oxygen_per_unit_mass_in_sea_water"})
    
    df.rename(columns={"%pressure(dbar)":"sea_water_pressure","pressure(dbar)":"sea_water_pressure","temperature(C)":"sea_water_temperature","salinity":"sea_water_salinity"}, inplace=True)
    df = df.set_index("sea_water_pressure")

    ds = xr.Dataset.from_dataframe(df)
    

    ds["time"] = measurement_time
    ds["latitude"]  = measurement_lat
    ds["longitude"]  = measurement_lon
    
    #profile_nr = int(list(pd.read_table(i,sep="[:, ]",nrows=0,engine="python"))[4])
    profile_nr = int(str(meta.head().columns[3])[:-1])
    
    
    ds = ds.assign_coords(profile=profile_nr)
    ds = ds.expand_dims("profile")

    
    # joining files
    if first==True:
        buoy= ds
        first=False
    else:
        buoy=xr.concat([buoy,ds],dim = "profile")
        #concat gjør at det blir mye nan i temp og salinity, men det følger cf. hør med Steingod

print("Det tok", time.time()-start)

#La oss fikse litt metadata da
#legg til alle de her greiene: https://adc.met.no/node/4
#lager metadata:
buoy.time.attrs["standard_name"] = "time"
buoy.time.attrs["units"] = "seconds since 1970-01-01 00:00:00+0"# husk å faktisk endre til dette formatet
buoy["sea_water_temperature"].attrs["standard_name"] = "sea_water_temperature"
buoy["sea_water_temperature"].attrs["units"] = "celcius" # husk å bytte til kelvin?
buoy["sea_water_pressure"].attrs["standard_name"] = "sea_water_pressure"
buoy["sea_water_pressure"].attrs["units"] = "dbar"

if "dissolved_oxygen" in df.columns:
    buoy["moles_of_oxygen_per_unit_mass_in_sea_water"].attrs["standard_name"] = "moles_of_oxygen_per_unit_mass_in_sea_water"
    buoy["moles_of_oxygen_per_unit_mass_in_sea_water"].attrs["units"] = "umol/kg"

#global attributes
buoy.attrs["title"] = "Nan"
buoy.attrs["summary"] = "Nan"
buoy.attrs["keywords"] = "Nan"
buoy.attrs["keywords_vocabulary"] = "Nan"
buoy.attrs["featureType"] = "trajectoryProfile"

buoy.attrs["geospatial_lat_min"] = min(buoy.latitude.values)
buoy.attrs["geospatial_lat_max"] = max(buoy.latitude.values)
buoy.attrs["geospatial_lon_min"] = min(buoy.longitude.values)
buoy.attrs["geospatial_lon_max"] = max(buoy.longitude.values)

buoy.attrs["time_coverage_start"] = min(buoy.time.values)
buoy.attrs["time_coverage_end"] = max(buoy.time.values)

buoy.attrs["Conventions"] = "ACDD-1.3"
buoy.attrs["history"] = "Nan"
buoy.attrs["date_created"] = "Nan"
buoy.attrs["creator_type"] = "Nan"
buoy.attrs["creator_institution"] = "Nan"
buoy.attrs["creator_name"] = "Nan"
buoy.attrs["creator_email"] = "Nan"
buoy.attrs["creator_url"] = "Nan"
buoy.attrs["project"] = "Nan"
buoy.attrs["license"] = "Nan"


buoy

233 0
378 1
120 3
122 4
119 8
121 9
120 13
121 14
119 18
121 19
Det tok 0.17498469352722168


<xarray.Dataset>
Dimensions:                (profile: 10, sea_water_pressure: 378)
Coordinates:
  * sea_water_pressure     (sea_water_pressure) int64 6 8 10 12 ... 756 758 760
  * profile                (profile) int64 0 1 3 4 8 9 13 14 18 19
Data variables:
    sea_water_temperature  (profile, sea_water_pressure) float64 nan nan ... nan
    sea_water_salinity     (profile, sea_water_pressure) float64 nan nan ... nan
    time                   (profile) datetime64[ns] 2019-09-22T02:02:04.128000 ... 2019-09-24T00:02:00.960000
    latitude               (profile) float64 -145.2 -145.9 ... -146.3 -146.3
    longitude              (profile) float64 77.97 77.99 77.99 ... 78.0 77.99
Attributes:
    title:                Nan
    summary:              Nan
    keywords:             Nan
    keywords_vocabulary:  Nan
    featureType:          trajectoryProfile
    geospatial_lat_min:   -146.3062
    geospatial_lat_max:   -145.1673
    geospatial_lon_min:   77.9705
    geospatial_lon_max:   78.0027
    time_coverage_start:  2019-09-22T02:02:04.128000000
    time_coverage_end:    2019-09-24T00:02:00.960000000
    Conventions:          ACDD-1.3
    history:              Nan
    date_created:         Nan
    creator_type:         Nan
    creator_institution:  Nan
    creator_name:         Nan
    creator_email:        Nan
    creator_url:          Nan
    project:              Nan
    licence:              Nan

Info:
====
**infoen er forløpig basert på gjennomgang av bare en bøye + at jeg har sett litt på hva som står om de andre. Så den kan være ugenerell, oppdaterer fortløpende. **


Data fra bøyene inholder profiler med trykk, temp og salt. 
Hver profil har en .dat fil med en header som inneholder starttid for profilen, lat,long og hvor mange målinger fila inneholder. enhet for tid er ÅR DAG, der dag har desimaler som gir klokkeslett. dag begynner på 1, ikke null. 

Noen bøyer har også andre målinger: bøya jeg nå ser på, 114, har to fastmonterte microcatsensorer som måler på 5 og 6 meters dyp hvert kvarter. de ligger i filene som har micro i navnet og er mye større. 

Jobber i første omgang med å lage et program som henter data fra én profil og en bøye.


Lage et program som dumper dette til netCDF-CF. 


Tanker og Spørsmål:
============



Ser ut til at metadata og discover-metadata må hardcodes for å stemme med standarene for [metadata]() og [discover-metadata](https://adc.met.no/node/4)

Skal jeg bruke trajectoryProfile eller Profile

skal jeg inkludere mlinger fra f.eks microcat-sensorene 